<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Real-Time-Audio-Intercommunicator-(RTAI)" data-toc-modified-id="Real-Time-Audio-Intercommunicator-(RTAI)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><a>Real Time Audio Intercommunicator (RTAI)</a></a></span><ul class="toc-item"><li><span><a href="#The-sounddevice.Stream-class" data-toc-modified-id="The-sounddevice.Stream-class-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>The <a href="https://python-sounddevice.readthedocs.io/en/0.3.15/api/streams.html" target="_blank"><code>sounddevice.Stream</code></a> class</a></span></li><li><span><a href="#Transmitting-live-data-through-the-Internet" data-toc-modified-id="Transmitting-live-data-through-the-Internet-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Transmitting live data through the Internet</a></span></li><li><span><a href="#Some-socket-stuff-in-Python" data-toc-modified-id="Some-socket-stuff-in-Python-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Some <a href="https://docs.python.org/3/library/socket.html" target="_blank"><code>socket</code></a> stuff in Python</a></span><ul class="toc-item"><li><span><a href="#Array-declarations" data-toc-modified-id="Array-declarations-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Array declarations</a></span></li><li><span><a href="#NumPy's-types" data-toc-modified-id="NumPy's-types-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>NumPy's types</a></span></li></ul></li></ul></li></ul></div>

# Sounddevice
`sounddevice` is a [Python](https://www.python.org) [module](https://docs.python.org/3/tutorial/modules.html) available for Linux, OSX and MS Windows. `sounddevice` is a Python [wrapper](https://en.wikipedia.org/wiki/Wrapper_library) for the [PortAudio](http://www.portaudio.com) library, which allows us to handle [PCM](https://en.wikipedia.org/wiki/Pulse-code_modulation) [audio](https://en.wikipedia.org/wiki/Digital_audio) in these operating systems.

In [1]:
try:
    import sounddevice as sd  # https://python-sounddevice.readthedocs.io
except ModuleNotFoundError:
    import os
    os.system("pip3 install sounddevice --user")
    import sounddevice as sd

In [11]:
help(sd)

Help on module sounddevice:

NAME
    sounddevice - Play and Record Sound with Python.

DESCRIPTION
    API overview:
      * Convenience functions to play and record NumPy arrays:
        `play()`, `rec()`, `playrec()` and the related functions
        `wait()`, `stop()`, `get_status()`, `get_stream()`
    
      * Functions to get information about the available hardware:
        `query_devices()`, `query_hostapis()`,
        `check_input_settings()`, `check_output_settings()`
    
      * Module-wide default settings: `default`
    
      * Platform-specific settings:
        `AsioSettings`, `CoreAudioSettings`, `WasapiSettings`
    
      * PortAudio streams, using NumPy arrays:
        `Stream`, `InputStream`, `OutputStream`
    
      * PortAudio streams, using Python buffer objects (NumPy not needed):
        `RawStream`, `RawInputStream`, `RawOutputStream`
    
      * Miscellaneous functions and classes:
        `sleep()`, `get_portaudio_version()`, `CallbackFlags`,
        `C

In [26]:
# Get device's list
sd.query_devices()

> 0 Built-in Microphone, Core Audio (2 in, 0 out)
  1 Built-in Input, Core Audio (2 in, 0 out)
< 2 Built-in Output, Core Audio (0 in, 2 out)
  3 eqMac2, Core Audio (2 in, 2 out)

In [21]:
# Get information about how many devices are available and the default ones
sd.query_hostapis()

({'name': 'Core Audio',
  'devices': [0, 1, 2, 3],
  'default_input_device': 0,
  'default_output_device': 2},)

In [22]:
# Get information about default input device
sd.query_devices(0)

{'name': 'Built-in Microphone',
 'hostapi': 0,
 'max_input_channels': 2,
 'max_output_channels': 0,
 'default_low_input_latency': 0.0037868480725623582,
 'default_low_output_latency': 0.01,
 'default_high_input_latency': 0.013945578231292517,
 'default_high_output_latency': 0.1,
 'default_samplerate': 44100.0}

In [27]:
# Get information about default output device
sd.query_devices(2)

{'name': 'Built-in Output',
 'hostapi': 0,
 'max_input_channels': 0,
 'max_output_channels': 2,
 'default_low_input_latency': 0.01,
 'default_low_output_latency': 0.003696145124716553,
 'default_high_input_latency': 0.1,
 'default_high_output_latency': 0.013854875283446711,
 'default_samplerate': 44100.0}

## The [`sounddevice.Stream`](https://python-sounddevice.readthedocs.io/en/0.3.15/api/streams.html) class
A `sounddevice.Stream` object allows to simulteneous input and output PCM digital audio through [NumPy](https://numpy.org) arrays. The following parameters are available (all are optional). Summarizing:
1. `samplerate`: Sampling frequency (for both, input and output) in frames per second.
2. `blocksize`: Number of frames (single samples in the case of mono audio or tuples of samples in the case of multichannel audio, normally, stereo) passed to the callback function (see below). By default, `blocksize=0`, which means that the block size possiblely will have a variable size, depending on the host workload and the requested latency setting (see below).
3. `device`: Input and output devices.
4. `dtype`: The sample format of the [numpy.ndarray](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html) provided to the stream callback (see below).
5. `latency`: The desired latency ([time elapsed between an action that produces sound and the actual perception of that sound by a listener](https://wiki.linuxaudio.org/wiki/jack_latency_tests)) of the ADC converter in seconds. The special values `low` and `high` (being the latter one the default) select the default low and high latency, respectively. This parameter has only effect if `blocksize!=0`.
6. `extra_settings`: Used for host-API-specific input/output settings.
7. `callback`: Callback function, which has the signature:
```
callback(indata: ndarray, outdata: ndarray, frames: int, time: CData, status: CallbackFlags) -> None
```
    1. `indata` and `outdata`: Input and output buffer, respectively, as two-dimensional `numpy.ndarray` with one column per channel (i.e. with a shape of `(frames, channels)`) and with a data type specified by `dtype`. The output buffer contains uninitialized data and the callback is supposed to fill it with audio data, that will depend on the application. `
    2. `frames`: Number of frames in `indata` and `outdata`.
    3. `time`: Time-stamps of the first frame in `indata`, in `outdata`, and the time at which the callback function was called.
    4. `status`: Indicates if underflow or overflow conditions happened during the last call to the callbak function. An underflow happens when the audio device is consuming the data faster than it arrives to the audio buffer. An overflow happens when the audio device is consuming the data too slow and the audio buffer overflows. Typically, the underflow problem is much more frequent than the overflow problem.
8. `finished_callback`: User-supplied function which will be called when the stream becomes inactive.
9. `clip_off`: Set to `True` to disable [clipping](https://en.wikipedia.org/wiki/Clipping_(audio)).
10. `dither_off`: Set to `True` to disable [dithering](https://en.wikipedia.org/wiki/Dither).
11. `never_drop_input`: Set to `True` to request that, where possible, a full duplex stream will not discard overflowed input samples without calling the stream callback. This only works if `blocksize=0`.
12. `prime_output_buffers_using_stream_callback`: Set to `True` to call the stream callback to fill initial output buffers, rather than the default behavior of priming the buffers with zeros (silence).

In [83]:
try:
    import numpy as np
except ModuleNotFoundError:
    import os
    os.system("pip3 install numpy --user")
    import numpy as np
import time
import sys
try:
    import psutil
except ModuleNotFoundError:
    import os
    os.system("pip3 install psutil --user")
    import psutil

def record_and_play(indata, outdata, frames, time, status):
    outdata[:] = indata
    print(f"measured buffer latency = {1000*(time.outputBufferDacTime - time.inputBufferAdcTime):3.3f} mili-seconds;",
        f"current time = {time.currentTime:8.2f} seconds;",
        f"CPU usage = {psutil.cpu_percent():4.2f}", end='\r')

def run(frames_per_second, frames_per_block):
    with sd.Stream(samplerate=frames_per_second,
                   blocksize=frames_per_block,
                   dtype=np.int16,
                   channels=2,
                   callback=record_and_play):
        print(f"ideal (minimum possible) buffer latency = {1000*(frames_per_block/frames_per_second):3.3f} mili-seconds")
        while True:
            time.sleep(1)

# Typical configuration
run(frames_per_second = 44100, frames_per_block = 1024)

ideal (minimum possible) buffer latency = 23.220 mili-seconds


KeyboardInterrupt: 

In [84]:
# If the sampling frequency is decreased, the buffer latency is increased (ideally) proportionally
run(frames_per_second = 22050, frames_per_block = 1024)

ideal (minimum possible) buffer latency = 46.440 mili-seconds


KeyboardInterrupt: 

In [ ]:
# If the buffer size is smaller, the buffer latency is also smaller (ideally) proportionally
run(frames_per_second = 44100, frames_per_block = 512)